# Experiments
现在是时候将所有内容整合起来，并在预训练数据集上训练（小型）语言模型了。



## 7.1 如何运行实验并提交成果
要真正理解 Transformer 架构组件背后的设计原理，最好的方法就是亲自修改并运行它。动手实践是无可替代的。

为此，能够快速、一致地进行实验，并记录下所做的一切至关重要。为了实现快速实验，我们将使用一个小型模型（1700万参数）和一个简单的数据集（TinyStories）进行多次小规模实验。为了确保实验的一致性，你需要以系统化的方式对各个组件进行消融分析，并调整超参数。同时，为了保留实验记录，我们要求你提交一份实验日志，以及每次实验对应的学习曲线。

为了能够提交损失曲线，请务必定期评估验证集上的损失，并记录训练的步数和实际耗时（wallclock time）。你可能会发现像 Weights and Biases 这样的日志记录工具非常有帮助。

#### 问题（experiment_log）：实验日志记录
为你的训练和评估代码建立实验跟踪基础设施，以便能够根据梯度更新步数（gradient steps）和实际耗时（wallclock time）来追踪实验过程和损失曲线。

交付内容：实验的日志记录基础设施代码，以及一份实验日志（即本节后续作业问题中你所尝试的所有内容的记录文档）。

## 7.2 TinyStories
我们将从一个非常简单的数据集（TinyStories；Eldan 和 Li，2023）开始，在该数据集上模型训练速度很快，同时我们也能观察到一些有趣的行为。获取该数据集的说明请参见第1节。

#### **超参数调优**
我们会为你提供一些基本的超参数作为起点，然后请你通过实验找出其他超参数的良好设置。

* vocab_size（词汇表大小）: 10000。典型的词汇表大小通常在几万到几十万之间。你可以尝试调整该值，观察词汇表大小对模型行为的影响。
* context_length（上下文长度）: 256。像 TinyStories 这样的简单数据集可能不需要很长的序列长度，但在后续使用 OpenWebText 数据时，你可能需要调整该值。尝试改变上下文长度，观察其对每次迭代的运行时间和最终困惑度（perplexity）的影响。
* d_model（模型维度）: 512。这比许多小型 Transformer 论文中常用的 768 维略小，但可以加快训练速度。
* d_ff（前馈网络维度）: 1344。这大约是 d_model 的 8/3 倍，同时是 64 的倍数，有利于 GPU 性能优化。
* RoPE theta 参数（Θ）: 10000。
* 层数与注意力头数: 4 层，16 个注意力头。这样的配置总共约有 1700 万个非嵌入参数，是一个相对较小的 Transformer 模型。
* 总处理 token 数: 327,680,000（你的 batch size × 总训练步数 × 上下文长度 应大致等于该值）。

**你需要通过一些试错，为以下超参数找到合适的默认值：**

* 学习率（learning rate）
* 学习率预热步数（learning rate warmup）
* 其他 AdamW 超参数（β₁, β₂, ϵ）
* 权重衰减（weight decay）
>这些超参数的典型取值可参考 Kingma 和 Ba [2015] 的论文。

整合所有内容
现在，你可以将所有部分整合起来：获取一个训练好的 BPE 分词器，对训练数据集进行分词，并在你编写的训练循环中运行模型。
重要提示：如果你的实现正确且高效，上述超参数在 1 块 **H100 GPU** 上的运行时间应大约为 30–40 分钟。如果你的运行时间显著更长，请检查你的数据加载、模型保存或验证损失计算代码是否存在性能瓶颈，并确保你的实现已正确地进行了批处理（batched）。

#### 问题（learning_rate）：调优学习率（4 H100 小时）
学习率是最关键的超参数之一，需要仔细调优。基于你已搭建的基础模型，请回答以下问题：

*  对学习率进行超参数搜索，报告不同学习率下的最终损失（如果优化器发散，请注明“发散”情况）。
*  经验法则认为，最优学习率处于“稳定性的边缘”（at the edge of stability）。请探究学习率开始发散的临界点与其最佳学习率之间的关系。


#### BPE分词

*假设在2.5已经完成预分词，数据保存在`.\out\gpt2_merges.txt` 和`.\out\gpt2_vocab.json`*

>vocab_size 太小，重新训练
现在加载整个训练集分词，将其映射为token格式，并保存.

In [ ]:
%run "./Experiments/loadTokens.py"

开始编码训练数据...


🔄Encoding: 100%|██████████| 222685/222685 [04:38<00:00, 798.38chunk/s]


✅训练数据加载完成，tokens=1282752668
💾训练数据编码完成并已保存到./out/encoded_ids_train.pkl

开始编码验证数据...


🔄Encoding: 100%|██████████| 2250/2250 [00:03<00:00, 724.01chunk/s]


✅验证数据加载完成，tokens=12956510
💾验证数据编码完成并已保存到./out/encoded_ids_valid.pkl



In [1]:
%run "./Experiments/train.py"

✅模型加载完成

📅日志时间戳: 20260210_131329
💻训练设备: cuda:0
验证间隔批次: 10 epochs
✔️从 checkpoint 恢复: epoch=9, step_in_epoch=4999, global_step=49998
🚀开始训练...



🔄Epoch 9: 100%|██████████| 5000/5000 [10:03<00:00,  8.29step/s, loss=1.066823, 学习率=0.000010]


Epoch 9 completed with loss: 1.066823
💾日志已保存至📁./Experiments/logs\training_log_20260210_040024.txt

🛠️保存 checkpoint...
💾Checkpoint 成功保存 epoch 9 至文件📁 ./Experiments/data/checkpoints\model_epoch_9_20260210_040024.pth
用户主动中断训练. Checkpoint 保存至📁 ./Experiments/data/checkpoints\interrupt_epoch_9_step_4999_20260210_040024.pth



KeyboardInterrupt: 

#### 自动实现中断恢复功能

In [1]:
%run "./Experiments/train.py"

✅模型加载完成

📅日志时间戳: 20260210_205014
💻训练设备: cuda:0
验证间隔批次: 40 epochs
✔️从 checkpoint 恢复: epoch=13, step_in_epoch=4635, global_step=79634
🚀开始训练...



🔄Epoch 13: 100%|██████████| 364/364 [00:45<00:00,  7.93step/s, loss=1.095762, 学习率=0.000010]


Epoch 13 completed with loss: 1.095762
💾日志已保存至📁./Experiments/logs\training_log_20260210_040024.txt



🔄Epoch 14: 100%|██████████| 5000/5000 [10:11<00:00,  8.17step/s, loss=0.977786, 学习率=0.000010]


Epoch 14 completed with loss: 0.977786
💾日志已保存至📁./Experiments/logs\training_log_20260210_040024.txt

🛠️保存 checkpoint...
💾Checkpoint 成功保存 epoch 14 至文件📁 ./Experiments/data/checkpoints\model_epoch_14_20260210_040024.pth



...

Epoch 38 completed with loss: 0.923482
💾日志已保存至📁./Experiments/logs\training_log_20260210_040024.txt



🔄Epoch 39: 100%|██████████| 5000/5000 [10:03<00:00,  8.28step/s, loss=0.935194, 学习率=0.000010]


Epoch 39 completed with loss: 0.935194
💾日志已保存至📁./Experiments/logs\training_log_20260210_040024.txt

🛠️保存 checkpoint...
💾Checkpoint 成功保存 epoch 39 至文件📁 ./Experiments/data/checkpoints\model_epoch_39_20260210_040024.pth

🔍开始验证...


🔍验证: 404882step [4:29:31, 25.04step/s]


✅验证完成

训练结束✅. Final checkpoint 已保存至:  ./Experiments/data/checkpoints\model_final_20260210_040024.pth


#### 推理


In [2]:
%run "./Experiments/predict.py"

Input Text:
 Once upon a time, there was a pretty girl named Lily.One day, Lily’s mom asked her to help cook dinner.

Output Text:
Once <unk> a time, there was a <unk>tty girl <unk> Lily.One day, Lily<unk><unk><unk>s mom asked her to <unk> cook <unk>ner. softlyirp grandchi hou marble� wonde admiriGo anyw bracel fasc mild bright ug boxes balancin Lea forehea excite JoSp yelling ne hopi meat baseb ta squirrel bu pencil overjo aim outdoy laundryshe An rocgh dollarHowev pebb turninReallflicky pours grant bought drivin puzzle thirs yo cream zooke porc� sandcastle Jaco ball sens imagin dee unkn spray al wand brushed sinc means under dran screaming hair brotheEw veterinaria trum nati studi Lilly sme On floati wast CharFred sec Ball pigs ma� Slowly suffer disp amazement frighte writesbbit traveled strhen if pours poss pas~ sea pizza any Br!" heel ordinary cele yester acorn outdoors strawberrie pizza sme Ruby Bub ow Friend fell eac attention myse lady recomm how bunny gav heale hospi knigh


## 7.3 消融实验与架构修改
理解 Transformer 的最佳方式就是实际修改它，并观察其行为变化。现在我们将进行一些简单的消融实验和架构修改。




### 7.3.1 消融实验：层归一化（layer normalization）

#### 问题（layer_norm_ablation）：移除 RMSNorm 并训练

>训练耗时过长，直接给出答案
* 前向传播崩溃，激活值方差爆炸，注意力机制失效，梯度饱和。
* 注意力分数极端化，容易出现**梯度不稳定**现象，梯度反向传播失效(梯度消失或者爆炸)

#### 问题（pre_norm_ablation）：实现并训练后归一化模型

在原先的模型中使用的是**预归一化**:

Transfomer 块为：
$$z = x + \text{MultiHeadedSelfAttention}(\text{RMSNorm}(x))$$
 
 
$$y = z + \text{FFN}(\text{RMSNorm}(z)).$$


后归一化修改了一下，将残差网络放 RMSNorm 中:
$$z = \text{RMSNorm}(x + \text{MultiHeadedSelfAttention}(x))$$
 
 
$$y  = \text{RMSNorm}(z + \text{FFN}(z)).$$

这里直接说明原因：

后归一化（Post-LayerNorm，或称经典Transformer）效果更差，其根本原因在于它破坏了训练过程中至关重要的梯度流，尤其是在深层网络中。

严谨一点，从数学上分析：

* 记第 $l$ 层的输入为 $\mathbf{x}^{l-1}$ 输出为 $\mathbf{x}_{l}$
* 每个 Transformer 块的核心变换（自注意力 + 前馈网络）记为函数 $\mathbf{F}_l(⋅)$，其参数为 $\theta_l$
* LayerNorm（或 RMSNorm）函数记为 $\text{LN}(\cdot)$.假设它执行如下操作（简化）：
  $$\text{LN}(\mathbf{x}) = \frac{\mathbf{x} - \mu}{\sigma} \odot \boldsymbol{\gamma} + \boldsymbol{\beta}$$
  其中 $\mu, \sigma$ 是标量，$\boldsymbol{\gamma}, \boldsymbol{\beta}$ 是可学习的缩放和偏移。
 
####  **前向传播公式**

 **后归一化（Post-LN）**
对于第 $l$ 层：
$$\mathbf{x}^l = \text{LN}\left( \mathbf{x}^{l-1} + \mathbf{F}_l(\mathbf{x}^{l-1}) \right)$$

 **预归一化（Pre-LN）**
对于第 $l$ 层：
$$\mathbf{x}^l = \mathbf{x}^{l-1} + \mathbf{F}_l\left( \text{LN}(\mathbf{x}^{l-1}) \right)$$


#### 梯度反向传播的数学分析

损失函数 $\mathcal{L}$ 对第 $k$ 层参数 $\boldsymbol{\theta}_k$ 的梯度：
$$
\frac{\partial \mathcal{L}}{\partial \boldsymbol{\theta}_k}
= \frac{\partial \mathcal{L}}{\partial \mathbf{x}^L} 
   \cdot \frac{\partial \mathbf{x}^L}{\partial \mathbf{x}^{L-1}} 
   \cdots \frac{\partial \mathbf{x}^{k+1}}{\partial \mathbf{x}^k} 
   \cdot \frac{\partial \mathbf{x}^k}{\partial \boldsymbol{\theta}_k}
$$
其中 $\mathbf{x}^L$ 是最后一层的输出。

定义层间雅可比矩阵：
$$
\mathbf{J}_l = \frac{\partial \mathbf{x}^l}{\partial \mathbf{x}^{l-1}} \in \mathbb{R}^{d \times d}
$$
则梯度传播可写作：
$$
\frac{\partial \mathcal{L}}{\partial \mathbf{x}^k} 
= \frac{\partial \mathcal{L}}{\partial \mathbf{x}^L} \cdot \mathbf{J}_L \cdot \mathbf{J}_{L-1} \cdots \mathbf{J}_{k+1}
$$

#### 计算两种架构的 $\mathbf{J}_l$

##### **后归一化（Post-LN）的雅可比**
令：
$$\mathbf{u}_l = \mathbf{x}^{l-1} + \mathbf{F}_l(\mathbf{x}^{l-1})$$
则：
$$\mathbf{x}^l = \text{LN}(\mathbf{u}_l)$$

求导：
$$
\mathbf{J}_l^{\text{post}} 
= \frac{\partial \mathbf{x}^l}{\partial \mathbf{x}^{l-1}}
= \frac{\partial \text{LN}(\mathbf{u}_l)}{\partial \mathbf{u}_l} 
   \cdot \frac{\partial \mathbf{u}_l}{\partial \mathbf{x}^{l-1}}
= \mathbf{D}_l \cdot \left( \mathbf{E} + \frac{\partial \mathbf{F}_l(\mathbf{x}^{l-1})}{\partial \mathbf{x}^{l-1}} \right)
$$
其中 $\mathbf{D}_l = \frac{\partial \text{LN}(\mathbf{u}_l)}{\partial \mathbf{u}_l}$，$\mathbf{E}$ 是单位矩阵。

* $ \frac{\partial \mathbf{F}_l(\mathbf{x}^{l-1})}{\partial \mathbf{x}^{l-1}}$ 是核心变换的雅可比，对于包含自注意力和前馈网络的复杂函数，其谱范数通常大于 1，尤其是当初始化权重较大或激活值方差较大时。
* $\mathbf{D}_l$ 是 LayerNorm 的雅可比，它是一个缩放矩阵，依赖于当前激活值 $\mathbf{u}_l$ 的统计量， $\mathbf{D}_l$ 的特征值大概在$\frac{\gamma}{\sigma}$ $\sigma$ 是 $\mathbf{u}_l$ 的标准差

**问题就出在 $\mathbf{D}_l$ 上 ，每个$\mathbf{J}_l^{\text{post}}$都可能偏离单位矩阵，少量层的偏离在连乘后会被指数放大。**


##### **预归一化（Pre-LN）的雅可比**

令：$$\mathbf{v}_l = \text{LN}(\mathbf{x}^{l-1})$$
则：$$\mathbf{x}^l = \mathbf{x}^{l-1} + \mathbf{F}_l(\mathbf{v}_l)$$
求导：
$$
\mathbf{J}_l^{\text{pre}} 
= \frac{\partial \mathbf{x}^l}{\partial \mathbf{x}^{l-1}}
= \mathbf{E} + \frac{\partial \mathbf{F}_l(\mathbf{v}_l)}{\partial \mathbf{v}_l} 
   \cdot \frac{\partial \text{LN}(\mathbf{x}^{l-1})}{\partial \mathbf{x}^{l-1}}
= \mathbf{E} + \mathbf{W}_l \cdot \mathbf{D}_{l-1}
$$
其中 $\mathbf{W}_l = \frac{\partial \mathbf{F}_l(\mathbf{v}_l)}{\partial \mathbf{v}_l}$， $\mathbf{D}_{l-1} = \frac{\partial \text{LN}(\mathbf{x}^{l-1})}{\partial \mathbf{x}^{l-1}}$。

* 主导项始终为恒等（单位）矩阵 $\mathbf{E}$

**两者对比发现，$\mathbf{D}_l$ 在训练初期是很容易不稳定的，对初始化的敏感度更高** 预归一化可以保证谱范数在1附近

### 7.3.2 消融实验：位置嵌入

#### 问题（no_pos_emb）：实现无位置嵌入（NoPE）
将你使用RoPE的Transformer实现修改为完全移除位置嵌入信息，观察会发生什么。
交付成果：一条对比RoPE与NoPE性能的学习曲线。

### 7.3.3 消融实验：SwiGLU 与 SiLU
接下来，我们将遵循 Shazeer [2020] 的方法，通过比较使用 SwiGLU 的前馈网络与仅使用 SiLU 激活函数但不包含门控线性单元（GLU）的前馈网络的性能，来测试前馈网络中门控机制的重要性
$$\mathrm{FFN}_{\mathrm{SiLU}}(x) = W_2 \mathrm{SiLU}(W_1 x) $$


$$\text{FFN}_{\text{SwiGLU}}(x)=W_2(\text{SiLU}(W_1x)⊙W_3x)$$

#### **为什么SwiGLU效果更好？**
* **门控机制是核心**：SwiGLU的本质是一个可学习的、动态的空间门。Swish(xW) 这个门控信号会对第二个投影 xV 的结果进行逐元素的缩放，让网络能学会在通道维度上精细化控制哪些信息应该被增强、哪些应该被抑制。这种能力在理解复杂语义和长程依赖时非常关键。
* **与Transformer的协同**：Transformer的核心是注意力机制，它负责融合信息。而FFN层（包含激活函数）负责变换和应用非线性到每个位置上。SwiGLU更强的非线性变换能力，与注意力机制形成了更有效的互补。
* **实践中的权衡**：由于SwiGLU增加了参数，为了进行公平比较，在实际使用时通常会减少隐层维度 d_ff。例如，如果原来用SiLU的FFN维度是 4*d_model，改用SwiGLU时可能会设为 (8/3)*d_model 或 2*d_model，以使两层线性层的总参数量大致相当。



## 7，4 OpenWebText
我们现在将转向一个更为标准的、由网络爬取数据构建的预训练数据集。OpenWebText [Gokaslan et al., 2019] 的一个小样本也以单个文本文件的形式提供：参见第1节了解如何访问该文件。

**注意：你可能需要为此实验重新调整你的超参数，例如学习率或批量大小。**



#### 问题（main_experiment）：在OWT上的实验（3个H100小时）
使用与TinyStories相同的模型架构和总训练步数，在OpenWebText上训练你的语言模型。该模型表现如何？

交付成果：你的语言模型在OpenWebText上的学习曲线。描述与TinyStories相比损失值的差异——我们应如何解释这些损失？

交付成果：从OpenWebText语言模型生成的文本，格式与TinyStories的输出相同。这段生成文本的流畅性如何？尽管我们使用了与TinyStories相同的模型和计算预算，为什么输出质量更差？